In [12]:
# read and clean up the log file
fh = open("userlog.log")
text = fh.readlines()
clean = []
listOfLines = []

for line in text:
    lineClean1 = line.replace('\n', '')
    lineClean2 = lineClean1.replace('\t', '')
    
    clean.append(lineClean2)
    
for line in clean:
    lineSplit = line.split()
    
    listOfLines.append(lineSplit)
    
dictOfUsers = {}

# parse and organize all of the data in each line
for line in listOfLines:
    user = line[4]
    
    if user not in dictOfUsers:
        x = {}
        dictOfUsers[user] = x

    date = line[0]
    time = line[1]
    activity = line[2]
    server = line[3]
    
    if date not in dictOfUsers[user]:
        timeDict = {}
        dictOfUsers[user][date] = timeDict
    
    if time not in dictOfUsers[user][date]:
        actServer = []
        dictOfUsers[user][date][time] = actServer
    
    dictOfUsers[user][date][time].append(activity)
    dictOfUsers[user][date][time].append(server)

fh.close()

In [13]:
# define a function to count the number of suspicious days for a user
def sus(user):
    susCount = 0
    
    for day in dictOfUsers[user]:
        sus = False
        logins = 0
        
        for time in dictOfUsers[user][day]:
            if sus == True:
                break
            
            if time >= "00:00:00" and time <= "05:00:00" and dictOfUsers[user][day][time][0] == "login":
                sus = True
                
            if dictOfUsers[user][day][time][0] == "login":
                logins += 1
                
        if sus == True:
            susCount += 1
            continue
            
        if logins > 5:
            susCount +=1
            
    return susCount

In [14]:
# define a function to count the number of irresponsible days for a user
def irr(user):
    irrCount = 0
    
    for day in dictOfUsers[user]:
        logins = 0
        logouts = 0
        
        for time in dictOfUsers[user][day]:
            if dictOfUsers[user][day][time][0] == "login":
                logins += 1
            
            else:
                logouts += 1
            
        if logins > logouts:
            irrCount +=1
            
    return irrCount

In [15]:
# define a function to count the number of system glitch days for a user
def glitch(user):
    glitchCount = 0
    
    for day in dictOfUsers[user]:
        logins = 0
        logouts = 0
        
        for time in dictOfUsers[user][day]:
            if dictOfUsers[user][day][time][0] == "login":
                logins += 1
            
            else:
                logouts += 1
            
        if logouts > logins:
            glitchCount +=1
            
    return glitchCount

In [16]:
# define a function to count the number of unique domains and to write all of them into a separate text file
def domains():
    domainDict = {}
    
    for user in dictOfUsers:
        userSplit = user.split("@")
        domain = userSplit[1]
        
        if (domain not in domainDict):
            domainDict[domain] = 1
            
        else:
            count = domainDict[domain]
            count += 1
            domainDict[domain] = count
    
    f = open("domain_report.txt", "w")
    f.write("==============================\n")
    f.write(f"=== DOMAIN COUNT ({len(domainDict)} DOMAINS) ===\n")
    f.write("==============================\n")
    
    for domain in domainDict:
        f.write(f"{domain}\t\t{domainDict[domain]}\n")

    f.close()

In [17]:
# build the domain report
domains()

In [20]:
# build the suspicious behavior report
susUsers = []
totalSus = 0

for user in dictOfUsers:
    totalSus += sus(user)
    
    if sus(user) > 0:
        susUsers.append(user)
        
susUsers.sort()

fs = open("suspicious_report.txt", "w")
fs.write("==============================\n")
fs.write(f"=== SUSPICIOUS BEHAVIOR ({totalSus} CASES) ===\n")
fs.write("==============================\n")

for user in susUsers:
    fs.write(f"{user}\t{sus(user)}\n")
    
    userDays = []
    
    for day in dictOfUsers[user]:
        userDays.append(day)
        
    userDays.sort()
    
    for day in userDays:
        fs.write(f"\tDATE: [{day}] ---\n")
        
        dayTimes = []
        
        for time in dictOfUsers[user][day]:
            dayTimes.append(time)
            
        dayTimes.sort()
        
        for time in dayTimes:
            fs.write(f"\t\t{time}\t{dictOfUsers[user][day][time][0]}\t{dictOfUsers[user][day][time][1]}\n")
            
fs.close()

In [21]:
# build the irresponsible behavior report
irrUsers = []
totalIrr = 0

for user in dictOfUsers:
    totalIrr += irr(user)
    
    if irr(user) > 0:
        irrUsers.append(user)
        
irrUsers.sort()

fi = open("irresponsible_report.txt", "w")
fi.write("==============================\n")
fi.write(f"=== IRRESPONSIBLE BEHAVIOR ({totalIrr} CASES) ===\n")
fi.write("==============================\n")

for user in irrUsers:
    fi.write(f"{user}\t{irr(user)}\n")
    
    userDays = []
    
    for day in dictOfUsers[user]:
        userDays.append(day)
        
    userDays.sort()
    
    for day in userDays:
        fi.write(f"\tDATE: [{day}] ---\n")
        
        dayTimes = []
        
        for time in dictOfUsers[user][day]:
            dayTimes.append(time)
            
        dayTimes.sort()
        
        for time in dayTimes:
            fi.write(f"\t\t{time}\t{dictOfUsers[user][day][time][0]}\t{dictOfUsers[user][day][time][1]}\n")
            
fi.close()

In [19]:
# build the system glitch report
glitchUsers = []
totalGlitch = 0

for user in dictOfUsers:
    totalGlitch += glitch(user)
    
    if glitch(user) > 0:
        glitchUsers.append(user)
        
glitchUsers.sort()

fg = open("glitch_report.txt", "w")
fg.write("==============================\n")
fg.write(f"=== GLITCH BEHAVIOR ({totalGlitch} CASES) ===\n")
fg.write("==============================\n")

for user in glitchUsers:
    fg.write(f"{user}\t{glitch(user)}\n")
    
    userDays = []
    
    for day in dictOfUsers[user]:
        userDays.append(day)
        
    userDays.sort()
    
    for day in userDays:
        fg.write(f"\tDATE: [{day}] ---\n")
        
        dayTimes = []
        
        for time in dictOfUsers[user][day]:
            dayTimes.append(time)
            
        dayTimes.sort()
        
        for time in dayTimes:
            fg.write(f"\t\t{time}\t{dictOfUsers[user][day][time][0]}\t{dictOfUsers[user][day][time][1]}\n")
            
fg.close()